In [1]:
# # IMPORTS
# astronomy
from astropy.io import fits
# plotting 
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.pyplot as plt
import matplotlib as mpl
# data 
import numpy as np
# file IO
import csv
# my python files 
import match
import v2_AGN_DataAndPlotting as adp
import v2_SED as SED

# Read Data

In [2]:
# open COSMOS 2020 catalog file and get data 
with fits.open(adp.path_cat+'\\COSMOS2020_CLASSIC_R1_v2.0_master.fits') as hdul20:
    data20 = hdul20[1].data
    cols20 = hdul20[1].columns

In [3]:
# get IRAC data (unit = 'uJy')
ch1_data20  = np.array(data20['SPLASH_CH1_FLUX']) # 3.5686 um
print('.', end='')
ch2_data20  = np.array(data20['SPLASH_CH2_FLUX']) # 4.5067 um
print('.', end='')
ch3_data20  = np.array(data20['SPLASH_CH3_FLUX']) # 5.7788 um
print('.', end='')
ch4_data20  = np.array(data20['SPLASH_CH4_FLUX']) # 7.9958 um 
print('.', end='')
# get ID
id_data20 = np.array(data20['ID_COSMOS2015'])
print('.', end='')

.....

In [4]:
# read file 
dataNew = adp.UnpackTable(adp.path_csv+'\\New_Lx_cat.csv')

# fix types
dataNew['ID']   = np.array(dataNew['ID'], dtype=int)
dataNew['z']    = np.array(dataNew['z'],  dtype=float)
dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale

# replace C-COSMOS 2016 for X-ray selection 
id_data16 = dataNew['ID']
# L0510_data16 = dataNew['Lx_05_10']
L0510_data16 = dataNew['Lx_hard_2_10']

C:\Users\polar\AppData\Local\Temp\ipykernel_11772\2168648500.py:7: RuntimeWarning: divide by zero encountered in log10
  dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_11772\2168648500.py:7: RuntimeWarning: invalid value encountered in log10
  dataNew['Lx_05_10']     = np.log10(np.array(dataNew['Lx_05_10'],     dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_11772\2168648500.py:8: RuntimeWarning: divide by zero encountered in log10
  dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale
C:\Users\polar\AppData\Local\Temp\ipykernel_11772\2168648500.py:8: RuntimeWarning: invalid value encountered in log10
  dataNew['Lx_hard_2_10'] = np.log10(np.array(dataNew['Lx_hard_2_10'], dtype=float)) # log scale


In [5]:
# get redshifts less than or equal to zMax
def GetRedshifts_lessEq(zMax, zDict, printInfo=True) : 
    # get mask of 0<z<zMax
    mask_lt = (zDict['z'] >= 0.0) & (zDict['z'] <= zMax)
    # make dict of z
    zDict_lt = {
        'ID' : zDict['ID'][mask_lt],
        'z'  : zDict['z'][mask_lt]
    }
    # print info
    if(printInfo):
        print('Number of redshifts <='+str(zMax)+':', len(zDict_lt['z']))
    # return dictionary of redshifts less than 
    return(zDict_lt)

# get redshifts
zDict = adp.ReadRedshifts(file=adp.path_cat+'\COSMOS_z_matches.csv')
zDict_le = GetRedshifts_lessEq(zMax=1,zDict=zDict)

Cropped:  ['ID', 'z']
Number of redshifts: 487252
Number of redshifts <=1: 221725


In [10]:
def GetAGNduoFromFile(file, printInfo=True):
    # read AGN IDs
    agnIDs_file = adp.ReadFile(file)
    # convert agnIDs into a dictionary of numpy arrays 
    agnIDs_dict = { 
        'MIR'  : np.array(agnIDs_file[0], dtype=int), 
        'XRAY' : np.array(agnIDs_file[1], dtype=int) 
        }
    # print info
    if(printInfo) :
        print('Number of MIR AGN:\t',    len(agnIDs_dict['MIR']))
        print('Number of Xray AGN:\t',   len(agnIDs_dict['XRAY']))
    # return dict
    return(agnIDs_dict)

# get AGNs
agnIDs = GetAGNduoFromFile(adp.path_csv+'\\Lx44_AGNid_Xray_MIR.csv')

Cropped:  ['Row 1 = MIR selected AGN (Donley et al, 2012) with id (n=1470). Row 2 = X-ray selected AGN (Lx > 10^44 erg/s) with id and redshift (n=869). ']
Number of MIR AGN:	 1470
Number of Xray AGN:	 869


In [11]:
def GetAGNduo_withZ(agnIdDict, zDict, printInfo=True) : 
    # match IDs between AGN and available redshifts
    key_agnID_MIR,  key_z = match.match(agnIdDict['MIR'],  zDict['ID'])
    key_agnID_XRAY, key_z = match.match(agnIdDict['XRAY'], zDict['ID'])
    # create dict of AGN with redshfts
    agnIDs_dict_z = {
        'MIR'  : agnIdDict['MIR'][key_agnID_MIR],
        'XRAY' : agnIdDict['XRAY'][key_agnID_XRAY]
    }
    # create dict of AGN withhout redshifts
    noMIRz = np.ones(agnIdDict['MIR'].size, dtype=bool)
    noXrayz = np.ones(agnIdDict['XRAY'].size, dtype=bool)
    noMIRz[key_agnID_MIR] = False   # set match to False
    noXrayz[key_agnID_XRAY] = False
    agnIDs_dict_zNO = {
        'MIR'  : agnIdDict['MIR'][noMIRz],
        'XRAY' : agnIdDict['XRAY'][noXrayz]
    }
    # print info
    if(printInfo) :
        print('Number of MIR AGN with z:\t',    len(agnIDs_dict_z['MIR']) )
        print('Number of Xray AGN with z:\t',   len(agnIDs_dict_z['XRAY']))
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('Number of MIR AGN without z:\t',    len(agnIDs_dict_zNO['MIR']) )
        print('Number of Xray AGN without z:\t',   len(agnIDs_dict_zNO['XRAY']))
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('Total MIR AGN:\t',    len(agnIDs_dict_z['MIR']) +len(agnIDs_dict_zNO['MIR']) )
        print('Total XRAY AGN:\t',   len(agnIDs_dict_z['XRAY'])+len(agnIDs_dict_zNO['XRAY']))
    # return dict 
    return(agnIDs_dict_z, agnIDs_dict_zNO)

# get AGN with z<=1 and z>1
agnIDs_le, agnIDs_leNO = GetAGNduo_withZ(agnIDs, zDict_le)

Number of MIR AGN with z:	 199
Number of Xray AGN with z:	 57
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Number of MIR AGN without z:	 1271
Number of Xray AGN without z:	 812
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Total MIR AGN:	 1470
Total XRAY AGN:	 869


In [12]:
def TwoToTri(irID,xrID,printInfo=True):
    # match agn ids to get overlap 
    key_ir, key_xr = match.match(irID, xrID)
    # initialize
    mask_ir = np.ones(irID.size, dtype=bool)
    mask_xr = np.ones(xrID.size, dtype=bool)
    # set matched index to false
    mask_ir[key_ir] = False
    mask_xr[key_xr] = False
    # remove overlaped xray selected agn from ir selected list
    irID_exc = irID[mask_ir]
    # remove overlaped ir selected agn from xray selected list 
    xrID_exc = xrID[mask_xr]
    # create list of ir/xray match
    ixrID_inc = irID[key_ir]
    # create and return dict of list of agn groups
    idDict = {
        'RED' : irID_exc,
        'GRN' : ixrID_inc,
        'BLU' : xrID_exc
    }
    if(printInfo):
        # print info 
        print('Number of IR only:\t',   len(idDict['RED']))
        print('Number of Xray only:\t', len(idDict['BLU']))
        print('Number of matches:\t',   len(idDict['GRN']))
    return(idDict)

# split into RED/YEl/BLU
triID = TwoToTri(agnIDs['MIR'], agnIDs['XRAY'])
print('~~~')
triID_le = TwoToTri(agnIDs_le['MIR'], agnIDs_le['XRAY'])
print('~~~')
triID_leNO = TwoToTri(agnIDs_leNO['MIR'], agnIDs_leNO['XRAY'])

Number of IR only:	 1076
Number of Xray only:	 475
Number of matches:	 394
~~~
Number of IR only:	 174
Number of Xray only:	 32
Number of matches:	 25
~~~
Number of IR only:	 902
Number of Xray only:	 443
Number of matches:	 369


In [13]:
# write IDs of AGN by selection method
with open(adp.path_csv+'\\Lx44_AGNid_tricolor.csv', mode='w', newline='') as csv_file:
    # initialize csv
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # write to file 
    writer.writerow([
        'Row 1 = MIR exclusive AGN with id, redshift, and IRAC (n='+str(len(triID['RED']))+'). '+ 
        'Row 2 = MIR-X-ray inclusive AGN with id, redshift, and IRAC (n='+str(len(triID['GRN']))+'). '
        'Row 3 = X-ray exclusive AGN with id, redshift, and IRAC (n='+str(len(triID['BLU']))+'). '
    ])
    writer.writerow(triID['RED'])
    writer.writerow(triID['GRN'])
    writer.writerow(triID['BLU'])
    # confirm 
    print('List saved: Lx44_AGNid_tricolor.csv')

List saved: Lx44_AGNid_tricolor.csv
